In [59]:
import pandas as pd
import numpy as np
import sidetable 
import requests
from datetime import datetime, timedelta
import mysql.connector
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 
from geopy.geocoders import Nominatim

In [60]:
class Universidades():

    def __init__(self, duda):

        self.duda = duda


    
    def extraer_argentina(self):

        url_argentina = "http://universities.hipolabs.com/search?country=Argentina"
        response_argentina = requests.get(url = url_argentina)
        print(f"La solicitud ha tenido una respuesta con código {response_argentina.status_code}, y nos devuelve el mensaje {response_argentina.reason}")
        df_argentina = pd.DataFrame(response_argentina.json())
        return df_argentina
    

    def extraer_canada(self):

        url_canada = "http://universities.hipolabs.com/search?country=Canada"
        response_canada = requests.get(url = url_canada)
        print(f"La solicitud ha tenido una respuesta con código {response_canada.status_code}, y nos devuelve el mensaje {response_canada.reason}")
        df_canada = pd.DataFrame(response_canada.json())
        return df_canada
    

    def extraer_eeuu(self):

        url_eeuu = "http://universities.hipolabs.com/search?country=United States"
        response_eeuu = requests.get(url = url_eeuu)
        print(f"La solicitud ha tenido una respuesta con código {response_eeuu.status_code}, y nos devuelve el mensaje {response_eeuu.reason}")
        df_eeuu = pd.DataFrame(response_eeuu.json())
        return df_eeuu
    
    
    def limpieza(self, df):

        df.rename(columns = {col:col.replace("-", "_") for col in df.columns}, inplace = True)

        df.drop(["domains"], axis = 1, inplace = True)

        df= df.explode("web_pages")

        duplicados = df.duplicated(["name"]).sum()
        print(f"El dataframe tiene {duplicados} duplicados.")
        
        if duplicados > 0:
            df.drop_duplicates(["name"], inplace = True)
        print(f"Después de la limpieza, el dataframe tiene {df.duplicated(['name']).sum()} duplicados.")
        
        df.fillna(value= np.nan, inplace=True)
        print(f"El dataframe tiene un total de {df['state_province'].isnull().sum()} nulos en la columna 'state_province'.")

        df.fillna(value= "Unknown", inplace=True)
        print(f"Después de la limpieza, el dataframe tiene un total de {df['state_province'].isnull().sum()} nulos en la columna 'state_province'.")


        df["state_province"].replace("NV", "Nevada", inplace = True)
        df["state_province"].replace("TX", "Texas", inplace = True)
        df["state_province"].replace("IN", "Indianapolis", inplace = True)
        df["state_province"].replace("CA", "California", inplace = True)
        df["state_province"].replace("VA", "Virginia", inplace = True)
        df["state_province"].replace("NY", "New York", inplace = True)
        df["state_province"].replace("MI", "Michigan", inplace = True)
        df["state_province"].replace("GA", "Georgia", inplace = True)
        df["state_province"].replace("ND", "North Dakota", inplace = True)
        df["state_province"].replace("New York, NY", "New York", inplace = True)
        df["state_province"].replace("Ciudad Autónoma de Buenos Aires", "Buenos Aires", inplace = True)

        return df
    

    
    def sacar_coordenadas(self, df_argentina, df_canada, df_eeuu):

        provincias_argentina = df_argentina["state_province"].unique()
        provincias_canada = df_canada["state_province"].unique()
        provincias_eeuu = df_eeuu["state_province"].unique()
        provincias = list(provincias_argentina) + list(provincias_canada) + list(provincias_eeuu)
        latitudes = []
        longitudes = []

        for elem in provincias:

            geolocator = Nominatim(user_agent="Lidya")
            location = geolocator.geocode(elem)
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
    

        dicc_latlon = {"state_province": provincias, "latitude" : latitudes, "longitude": longitudes}
        df_latitud_longitud = pd.DataFrame(dicc_latlon)

        return df_latitud_longitud
    

    def unir_dataframes(self, df_argentina, df_canada, df_eeuu, df_latitud_longitud):

        df_paises = pd.concat([df_argentina, df_canada, df_eeuu], axis = 0)
        df_completo = df_paises.merge(df_latitud_longitud, on='state_province')

        return df_completo